In [1]:
from data_processing.audio_processor import AudioProcessor
from data_processing.spectrogram_processor import SpectrogramProcessor

In [2]:
from data_processing.audio_processor import AudioProcessor
import pandas as pd
from IPython.display import Audio
from pydub import AudioSegment
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np



In [3]:
# ------------- Example for running the reduce_noise function ------------- 
metadata_df = pd.DataFrame({
    "file_name": ["sample_audio.wav"],
    "duration": [5.0]
})
input_folder = 'data/cough_data/original_data/positive'

audio_processor = AudioProcessor(
    input_folder=input_folder,
    target_sample_rate=32000,  
    target_duration=15.0
)
#C:\Users\houma\RespiraCheck\ml\data\cough_data\original_data\ff2493d0-17da-4e51-b794-d10876b7048b.mp3
print("Processed Audio (Normalized for playback):")
audio = AudioSegment.from_file("data/cough_data/original_data/negative/0a1c8e24-81d7-404e-8a85-b3fd3999ecb5.wav",format="wav")
display(audio)
x = audio_processor.reduce_noise(audio)
display(x)

Processed Audio (Normalized for playback):


In [10]:
input_path = "data/cough_data/original_data/negative/0a1c8e24-81d7-404e-8a85-b3fd3999ecb5.wav"
output_path = "data/cough_data/testing/0a1c8e24-81d7-404e-8a85-b3fd3999ecb5.wav"
os.makedirs("data/cough_data/testing", exist_ok=True)

audio_processor.process_single_audio(input_path, output_path)


0

In [ ]:
 #  ------------------- example for running the remove_no_cough function -------------------
import os
import pandas as pd
from pydub import AudioSegment

# Define the input folder and sample input for testing
input_folder = 'data/cough_data/original_data'
sample_input_for_testing = 'data/cough_data/original_data/fee168b1-58b4-45ba-b2aa-f30d13e4af2f.mp3'

metadata_df = pd.DataFrame({
    "file_name": ["sample_audio.wav"],
    "duration": [5.0]
})

audio_processor = AudioProcessor(
    input_folder=input_folder,
    target_sample_rate=32000,  
    target_duration=15.0,       
    metadata_df=metadata_df
)

def convert_to_wav_pydub(file_path):
    if not file_path.lower().endswith('.wav'):
        wav_file_path = os.path.splitext(file_path)[0] + '.wav'
        if file_path.lower().endswith('.mp3'):
            sound = AudioSegment.from_mp3(file_path)
        elif file_path.lower().endswith('.flac'):
            sound = AudioSegment.from_file(file_path, format="flac")
        elif file_path.lower().endswith('.ogg'):
            sound = AudioSegment.from_file(file_path, format="ogg")
        else:
            raise ValueError(f"Unsupported file format: {file_path}")
        
        sound.export(wav_file_path, format="wav")
        
        return wav_file_path
    else:
        return file_path

sample_input_for_testing = convert_to_wav_pydub(sample_input_for_testing)
 
initial_size = sum(os.path.getsize(os.path.join(input_folder, f)) for f in os.listdir(input_folder))

# Process the audio file
audio_processor.remove_no_cough(sample_input_for_testing)
# Find final folder size
final_size = sum(os.path.getsize(os.path.join(input_folder, f)) for f in os.listdir(input_folder))
if final_size < initial_size:
    print("File sucessfully removed")

TypeError: AudioProcessor.__init__() got an unexpected keyword argument 'metadata_df'

In [ ]:
# -------- Example using the fbank feature extraction function ---------
def fbank(audio_path, samplerate=16000, winlen=0.025, winstep=0.01,
          nfilt=40, nfft=512, lowfreq=0, highfreq=None, preemph=0.97, 
          wintype='hamming', grayscale=False, save_image=False, image_path="fbank_image.png"):
        """Compute Mel-filterbank energy features and optionally convert to a grayscale image.
        
        :param audio_path: Path to the audio file.
        :param samplerate: Sample rate of the signal.
        :param winlen: Window length in seconds.
        :param winstep: Step size between windows in seconds.
        :param nfilt: Number of Mel filters.
        :param nfft: FFT size.
        :param lowfreq: Lowest frequency in Mel filters.
        :param highfreq: Highest frequency in Mel filters.
        :param preemph: Pre-emphasis factor.
        :param wintype: Window function type.
        :param grayscale: Whether to convert the filterbank to a grayscale image.
        :param save_image: Whether to save the grayscale image.
        :param image_path: File path to save the image.
        :return: Filterbank features (2D numpy array).
            """
        signal, samplerate = librosa.load(audio_path, sr=samplerate)
        
        highfreq = highfreq or samplerate / 2
        
        signal = np.append(signal[0], signal[1:] - preemph * signal[:-1])
        
        frame_length = int(winlen * samplerate)
        frame_step = int(winstep * samplerate)
        frames = librosa.util.frame(signal, frame_length=frame_length, hop_length=frame_step).T
        
        if wintype == 'hamming':
            window = np.hamming(frame_length)
        elif wintype == 'hann':
            window = np.hanning(frame_length)
        else:
            window = np.ones(frame_length)
        frames = frames.copy()

        frames *= window
        
        mag_frames = np.abs(np.fft.rfft(frames, n=nfft))
        pow_frames = (1.0 / nfft) * (mag_frames ** 2)
        
        mel_filters = librosa.filters.mel(sr=samplerate, n_fft=nfft, n_mels=nfilt, fmin=lowfreq, fmax=highfreq)
        fbank_features = np.dot(pow_frames, mel_filters.T)
        # linear scaling
        #fbank_features = np.where(fbank_features == 0, np.finfo(float).eps, fbank_features)

        # log scaling - makes differences more apparent
        fbank_features = librosa.power_to_db(fbank_features, ref=np.max)

        if grayscale or save_image:
            plt.figure(figsize=(4, 4))
            plt.imshow(fbank_features.T, cmap='gray', origin='lower', aspect='auto')
            plt.axis('off')
            if save_image:
                plt.savefig(image_path, bbox_inches='tight', pad_inches=0)
            plt.close()
        
        return fbank_features
fbank_features =fbank("data/cough_data/original_data/0a1c8e24-81d7-404e-8a85-b3fd3999ecb5.mp3", grayscale=True, save_image=True)
print(fbank_features)